In [107]:
import dask.dataframe as dd
import pandas as pd
import os

In [108]:
import dask
dask.config.set(scheduler="processes")

In [109]:
DATA_PATH = "data"
FILES_NAMES = os.listdir(os.path.join(DATA_PATH,"2022"))
FILES_NAMES

['finbra_balanco_patrimonial.csv',
 'finbra_despesas_orcamentarias.csv',
 'finbra_despesas_por_funcao.csv',
 'finbra_execucao_de_restos_a_pagar.csv',
 'finbra_execucao_de_restos_a_pagar_por_funcao.csv',
 'finbra_receitas_orcamentarias.csv',
 'finbra_variacoes_patrimoniais_DCA.csv']

In [110]:
to_float = lambda x: x.replace(",", ".")

# Balanco Patrimonial

In [113]:
year = "2022"
file_name = FILES_NAMES[0]
file_path = os.path.join(DATA_PATH,year,file_name)
bp = dd.read_csv(file_path, sep=";", encoding="latin-1", skiprows=3)
bp["Valor"] = bp["Valor"].apply(to_float, meta=pd.Series(name="Valor",dtype=str)).astype(float)
bp.head()

,Instituição,Cod.IBGE,UF,População,Coluna,Conta,Identificador da Conta,Valor
0,Prefeitura Municipal de Flores da Cunha - RS,4308201,RS,31352,31/12/2022,1.0.0.0.0.00.00 - Ativo,siconfi-cor_P1.0.0.0.0.00.00,3.703242e+08
1,Prefeitura Municipal de Flores da Cunha - RS,4308201,RS,31352,31/12/2022,1.1.0.0.0.00.00 - Ativo Circulante,siconfi-cor_P1.1.0.0.0.00.00,1.920448e+08
2,Prefeitura Municipal de Flores da Cunha - RS,4308201,RS,31352,31/12/2022,1.1.1.0.0.00.00 - Caixa e Equivalentes de Caixa,siconfi-cor_P1.1.1.0.0.00.00,5.701971e+07
3,Prefeitura Municipal de Flores da Cunha - RS,4308201,RS,31352,31/12/2022,1.1.1.1.0.00.00 - Caixa e Equivalentes de Caix...,siconfi-cor_P1.1.1.1.0.00.00,5.701971e+07
4,Prefeitura Municipal de Flores da Cunha - RS,4308201,RS,31352,31/12/2022,1.1.1.1.1.00.00 - Caixa e Equivalentes de Caix...,siconfi-cor_P1.1.1.1.1.00.00,5.701971e+07


In [115]:
bp.head()

,Instituição,Cod.IBGE,UF,População,Coluna,Conta,Identificador da Conta,Valor,Ano
0,Prefeitura Municipal de Flores da Cunha - RS,4308201,RS,31352,31/12/2022,1.0.0.0.0.00.00 - Ativo,siconfi-cor_P1.0.0.0.0.00.00,3.703242e+08,2022
1,Prefeitura Municipal de Flores da Cunha - RS,4308201,RS,31352,31/12/2022,1.1.0.0.0.00.00 - Ativo Circulante,siconfi-cor_P1.1.0.0.0.00.00,1.920448e+08,2022
2,Prefeitura Municipal de Flores da Cunha - RS,4308201,RS,31352,31/12/2022,1.1.1.0.0.00.00 - Caixa e Equivalentes de Caixa,siconfi-cor_P1.1.1.0.0.00.00,5.701971e+07,2022
3,Prefeitura Municipal de Flores da Cunha - RS,4308201,RS,31352,31/12/2022,1.1.1.1.0.00.00 - Caixa e Equivalentes de Caix...,siconfi-cor_P1.1.1.1.0.00.00,5.701971e+07,2022
4,Prefeitura Municipal de Flores da Cunha - RS,4308201,RS,31352,31/12/2022,1.1.1.1.1.00.00 - Caixa e Equivalentes de Caix...,siconfi-cor_P1.1.1.1.1.00.00,5.701971e+07,2022


# Despesas Orcamentarias

In [112]:
file_name = FILES_NAMES[1]
do = dd.read_csv(f"data/2022/{file_name}", sep=";", encoding="latin-1", skiprows=3)
do["Valor"] = do["Valor"].apply(to_float, meta=pd.Series(name="Valor",dtype=str)).astype(float)
bp.head()

,Instituição,Cod.IBGE,UF,População,Coluna,Conta,Identificador da Conta,Valor
0,Prefeitura Municipal de Sulina - PR,4126652,PR,3366,31/12/2013,1.0.0.0.0.00.00 - Ativo,siconfi-cor_P1.0.0.0.0.00.00,10178159.87
1,Prefeitura Municipal de Sulina - PR,4126652,PR,3366,31/12/2013,1.1.0.0.0.00.00 - Ativo Circulante,siconfi-cor_P1.1.0.0.0.00.00,500876.55
2,Prefeitura Municipal de Sulina - PR,4126652,PR,3366,31/12/2013,1.1.1.0.0.00.00 - Caixa e Equivalentes de Caixa,siconfi-cor_P1.1.1.0.0.00.00,398188.46
3,Prefeitura Municipal de Sulina - PR,4126652,PR,3366,31/12/2013,1.1.1.1.0.00.00 - Caixa e Equivalentes de Caix...,siconfi-cor_P1.1.1.1.0.00.00,398188.46
4,Prefeitura Municipal de Sulina - PR,4126652,PR,3366,31/12/2013,1.1.1.1.1.00.00 - Caixa e Equivalentes de Caix...,siconfi-cor_P1.1.1.1.1.00.00,398188.46
